In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from cell import Cell
import numpy
import matplotlib.pyplot as plt
from tqdm import tqdm

## first simulation growth without any abx as an integrity check

In [3]:
param = {"ribo_min"=19.3, "p_in"=1, "p_out"=1, "k_on"=1, "k_off"=1, "alpha"=1, "beta"=1, "gama"=1, "abx_env"=0, "t_start"=10000, "t_end"=-1}

SyntaxError: cannot assign to literal here. Maybe you meant '==' instead of '='? (1114488654.py, line 1)